In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix
import scipy.stats
import datetime
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

C:\Users\duskb\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\duskb\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


ModuleNotFoundError: No module named 'torch'

In [ ]:
sns.set_context("paper", font_scale=0.8, rc={"lines.linewidth": 1})
pd.options.display.max_rows = 4000

In [ ]:
df = pd.read_excel('botox database.xlsx')

# Wrangling

In [ ]:
df['Mirabegron'] = df['Mirabegron'].replace(['N', 'Y'], [0, 1])
df['Anticholinergics'] = df['Anticholinergics'].replace(['N', 'Y'], [0, 1])
df['DO'] = df['DO'].replace(['N', 'Y'], [0, 1])
df['BOO'] = df['BOO'].replace(['N', 'Y'], [0, 1])
df['Neuropathic'] = df['Neuropathic'].replace(['N', 'Y'], [0, 1])
df['SUI'] = df['SUI'].replace(['N', 'Y'], [0, 1])
df['UUI'] = df['UUI'].replace(['N', 'Y'], [0, 1])
df['Incontinence surgery'] = df['Incontinence surgery'].replace(['N', 'Y'], [0, 1])
df['Hysterectomy'] = df['Hysterectomy'].replace(['N', 'Y'], [0, 1])
df['Previous bladder outflow surgery'] = df['Previous bladder outflow surgery'].replace(['N', 'Y'], [0, 1])
df['PPI'] = df['PPI'].replace(['N', 'Y'], [0, 1])
df['Middle lobe?'] = df['Middle lobe?'].replace(['N', 'Y'], [0, 1])
df['CISC'] = df['CISC'].replace(['N', 'Y'], [0, 1])

In [ ]:
df.count()

In [ ]:
df['log PVR'] = np.log10(df['PVR']+1)

In [ ]:
df[['PVR', 'log PVR']]

In [ ]:
df['Dose <100'] = [1 if x<=100 else 0 for x in df['Dose']]

In [ ]:
df[['Dose', 'Dose <100']]

In [ ]:
df = df.fillna(0)

In [ ]:
dfF = df[df['Gender']==0]

In [ ]:
dfM = df[df['Gender']==1]

# Demographics

In [ ]:
def Counts(df, var):
    counts = df[var].value_counts()
    percent = df[var].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    return pd.DataFrame({'N': counts, '%': percent}) 

In [ ]:
df['Gender'].value_counts()

In [ ]:
df['Age'].describe()

In [ ]:
Counts(df, "Dose")

In [ ]:
sns.barplot(x="Dose", y="CISC", data=df, ci=None)

In [ ]:
Counts(dfF, "CISC")

In [ ]:
Counts(dfM, "CISC")

In [ ]:
cont=pd.crosstab(dfM['PPI'], dfM['CISC'])
cont

Of men without RARP, CISC rates were 68%

# ML model

In [ ]:
x = df[['Age', 'Gender', 'DO', 'BOO','Neuropathic','SUI','UUI','PVR','Hysterectomy','PPI','Prostate size (ml)', 'Dose <100', 'Mirabegron', 'Anticholinergics']]
y = df['CISC']

print(x.shape)
print(y.shape)

In [ ]:
df.isnull().values.any()

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
print(x_train)
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(x_train)
x_train = scalerX.transform(x_train)
x_test = scalerX.transform(x_test)

In [ ]:
x_test.shape

In [ ]:
class dataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y.values,dtype=torch.float32)
        self.length = self.x.shape[0]
 
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]  
    def __len__(self):
        return self.length

trainset = dataset(x_train,y_train)
testset = dataset(x_test,y_test)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self,input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,1)
        self.dropout = nn.Dropout(p=0.4)
    def forward(self,x):
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
learning_rate = 0.003
epochs = 600

# Model , Optimizer, Loss
model = Net(input_shape=x.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
torch.autograd.set_detect_anomaly(True)
#forward loop
train_losses, test_losses = [], []
accur = []
for i in range(epochs):
    for j,(x_train,y_train) in enumerate(trainloader):
    
        #calculate output
        output = model(x_train)
 
        #calculate loss
        running_loss = loss_fn(output,y_train.reshape(-1,1))
 
        #accuracy
        #predicted = model(torch.tensor(x,dtype=torch.float32))
        #acc = (predicted.reshape(-1).detach().numpy().round() == y).mean() 
        
        #backprop
        optimizer.zero_grad()
        running_loss.backward()
        optimizer.step()
    
    else:
        test_loss = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for j,(x_test,y_test) in enumerate(testloader):
                output = model(x_test)
 
                #calculate loss
                test_loss = loss_fn(output,y_test.reshape(-1,1))
 
                #accuracy
                #predicted = model(torch.tensor(x_test,dtype=torch.float32))
                #acc = (predicted.reshape(-1).detach().numpy().round() == y_test).mean() 
        
    if i%50 == 0:
        train_losses.append(running_loss)
        test_losses.append(test_loss)
        #accur.append(acc)

        print("Epoch: {}/{}.. ".format(i+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss),
              "Test Loss: {:.3f}.. ".format(test_loss))
              #"Test Accuracy: {:.3f}".format(acc))

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

In [ ]:
plt.plot(accur)

In [ ]:
#printing the accuracy
plt.plot(accur)
plt.title('Accuracy vs Epochs')
plt.xlabel('Accuracy')
plt.ylabel('loss')

In [ ]:
model.eval()
params = x_train[0]

with torch.no_grad():
    output = model.forward(params)

ps = torch.sigmoid(output)
print(scalerX.inverse_transform(x_train[0]))
print(ps)